## GovTech Text Classifier


In [153]:
import torch
from torchtext import data
from torchtext import datasets
from torchtext.data import Field, TabularDataset, LabelField
import random

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

question = Field(tokenize = 'spacy')
agency = LabelField(sequential=False)

fields = {"Question Asked": ("q", question), "Agency": ("s", agency)}

train_data, test_data = data.TabularDataset.splits(
        path="/content/drive/My Drive/gov", # the root directory where the data lies
        train='train_small.csv', test="test.csv",
        format='csv',
        fields=fields # if your csv header has a header, make sure to pass this to ensure it doesn't get processed as data!
        )
train_data, valid_data = train_data.split(random_state = random.seed(SEED))


In [154]:
train_data[0].__dict__.keys()
train_data[0].__dict__.values()

dict_values([['Hi', 'how', 'many', 'demerit', 'points', 'do', 'I', 'have'], 'SPF'])

In [155]:
valid_data[0].__dict__.keys()
valid_data[0].__dict__.values()

dict_values([['Other', 'employment', 'income', 'include', 'oversea', 'allowance'], 'IRAS IIT'])

In [156]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Build the vocabulary. 

In [157]:
MAX_VOCAB_SIZE = 25_000

question.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

agency.build_vocab(train_data)

# Next, we check the Agency.

In [158]:
print(agency.vocab.stoi)
len(agency.vocab)

defaultdict(<function _default_unk_index at 0x7fe5b021e048>, {'IRAS IIT': 0, 'ICA': 1, 'LTA': 2, 'ACRA': 3, 'SPF': 4, 'MINDEF': 5, 'SSG': 6, 'MOE': 7, 'IRAS GST': 8, 'AVA': 9})


10

# Set up the iterators.

In [159]:
BATCH_SIZE = 2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    sort = False, #don't sort test/validation data
    batch_size = BATCH_SIZE, 
    device = device)

# Build CNN model. Can try with other model like RNN, Bidirectional RNN etc

In [160]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        text = text.permute(1, 0)
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

#Hyper Parameters

In [161]:
INPUT_DIM = len(question.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [1,2]
OUTPUT_DIM = len(agency.vocab)
DROPOUT = 0.5
PAD_IDX = question.vocab.stoi[question.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

# Check the number of parameters

In [162]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 718,210 trainable parameters


# Load pre-trained embeddings.

In [163]:
pretrained_embeddings = question.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.1897,  0.0500,  0.1908,  ..., -0.3980,  0.4765, -0.1598],
        ...,
        [-0.5068, -0.8909,  0.0254,  ...,  1.6284,  0.1772, -0.7119],
        [ 0.3256, -0.1760,  0.5735,  ..., -0.1452,  0.6097, -2.6922],
        [ 0.0323,  0.2468,  0.0765,  ..., -0.0374,  0.3347, -0.1959]])

# Then zero the initial weights of the unknown and padding tokens.

In [164]:
UNK_IDX = question.vocab.stoi[question.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

# Use CrossEntropyLoss for loss function

In [165]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [167]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])

# Training Loop

In [168]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.q)
        
        loss = criterion(predictions, batch.s)
        
        acc = categorical_accuracy(predictions, batch.s)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

# Evaluation loop

In [169]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.q)
            
            loss = criterion(predictions, batch.s)
            
            acc = categorical_accuracy(predictions, batch.s)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [170]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# Train the model.

In [171]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'govtech-text-classifier.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 51s
	Train Loss: 1.391 | Train Acc: 55.48%
	 Val. Loss: 0.903 |  Val. Acc: 71.59%
Epoch: 02 | Epoch Time: 1m 1s
	Train Loss: 0.803 | Train Acc: 75.71%
	 Val. Loss: 0.726 |  Val. Acc: 78.31%
Epoch: 03 | Epoch Time: 1m 8s
	Train Loss: 0.586 | Train Acc: 82.33%
	 Val. Loss: 0.692 |  Val. Acc: 79.73%
Epoch: 04 | Epoch Time: 1m 1s
	Train Loss: 0.464 | Train Acc: 85.83%
	 Val. Loss: 0.712 |  Val. Acc: 80.33%
Epoch: 05 | Epoch Time: 1m 5s
	Train Loss: 0.350 | Train Acc: 89.27%
	 Val. Loss: 0.725 |  Val. Acc: 80.60%


#Run the model on the test dataset

In [172]:
model.load_state_dict(torch.load('govtech-text-classifier.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.674 | Test Acc: 80.59%


# Inference the model  with sample questions

In [175]:
import spacy
nlp = spacy.load('en')

def predict_class(model, sentence, min_len = 4):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [question.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    preds = model(tensor)
    max_preds = preds.argmax(dim = 1)
    return max_preds.item()

In [176]:
pred_class = predict_class(model, "who can claim for skill future")
print(f'Predicted class is: {pred_class} = {agency.vocab.itos[pred_class]}')

Predicted class is: 6 = SSG


In [177]:
pred_class = predict_class(model, "Why I cannot update my rent property")
print(f'Predicted class is: {pred_class} = {agency.vocab.itos[pred_class]}')

Predicted class is: 0 = IRAS IIT


In [178]:
pred_class = predict_class(model, "Extend passport")
print(f'Predicted class is: {pred_class} = {agency.vocab.itos[pred_class]}')

Predicted class is: 1 = ICA


In [179]:
pred_class = predict_class(model, "I lost my escooter")
print(f'Predicted class is: {pred_class} = {agency.vocab.itos[pred_class]}')

Predicted class is: 2 = LTA
